# Change Request Risk Assessment
#### Author: **John Heisler** - Senior AI Specialist, Financial Services, Snowflake
In this notebook, we're going to evaluate a new change request and determine a risk score for that change request resulting in target environment instability.

The notebook is written to allow deployment directly into anyone's environment.

## Hit the Nail, Don't Build the Hammer!
Ultimately, AI should drive commercial value. To that end, building, deploying, and maintaining the systems that underpin that commercial value need to be easy, efficient, and trusted. 
Enterprises that maximize efforts leveraging their differentiated domain knowledge and creativity will ultimately win the AI race. Every ounce of effort spent on managing/building/tuning AI is a distraction from delivering value and thus comes at a material opportunity cost. With Snowflake Cortex AI, we aim to maximize efforts wielding the power of AI, not building it.

## Use Case Commercial Value
Our solution drives **operational alpha** by maximizing uptime of production environments. This solution provides operational alpha in at least these four ways:
1. **System Uptime**: The systems critical to support our portfolio teams will be more stable and provide maximum value to our portfolio teams' performance.
2. **Regulatory Reporting**: Minimize need for reporting to external regulators about critical system downtime and its impact.
3. **Opportunity Cost**: Minimizing time spent conducting emergency maintenance which can be repurposed to focus on alpha-generating solutions.
4. **Opportunity Cost**: Minimize need for time-and-resource-intensive root cause analysis and these cross functional teams can focus on their primary responsibilities.

## Process Outline
Before we got here, using the harbinger_data_creation.ipynb notebook in this repo, we created and filled a change request table with some synthetic data (Fun Fact: You we use LLMs to do that-- check out the aforementioned notebook to see how).

1. Build a python function to house our prompt and accept a change request as context. this approach streamlines our code and decouples the prompt from our broader development, allowing for independent development on the prompt by domain experts.
2. Build a complete function to streamline passing of parameters to our complete function.
3. Generate Risk Scores and Reasonings with batch inference whereby we generate a risk score for change requests as they appear in our system.
4. Build a UI to allow the end user to interact with the AI.   
    * Summary Interface: leveraging the data that we persisted from our batch inference in step 3, render the results for review. This portion of the infercae is great for weekly reviews of change requests
    * Ad Hoc/Exploratory Interface: Allow an end user to pass a Change Request to an LLM of their choosing to explore how different LLMs perform on this task.


## Solution Differentiators
* **LLM fungibility**: We have a model garden right here in Snowflake-- no need to manage divergent infrastructure and no need for external calls which introduce risk into your system
* **Obeying the Laws of Data Physics**: We develop, perform inference, and host the UI in the same compute environment that the data resides-- eleminating bloated architectures and complexity associated with managing the data, inference, and UI in seperate infrastructures. 

## Art of the Possible
Here is some food for thought and hopefully inspiring enhancements for your deployment. 

* **Fine Tuning**: If we have tied incidents' root causes to change requests in the past, we could fine tune a model here in Snowflake with that data and the task that we're after. This has a couple of very interesting advantages. first, we would have a very specialize model that may perform this task very well because it has "experience" with what good and bad looks like. More operationally focused, we could maybe use a smaller model maximizing cost efficiencies.
* **More Context - Metadata**: we could offer much more context about what each column in the data set means and fully define the json structure that we're passing. This would inform the model on the meaning of each column rather than allowing it to come up with its best guess at the columns.
* **More Context - Target System Stability (Windshield)**: We could use another LLM upstream of the final risk inference to build a synopsis of the target system stability and status. For instance, it could state things like, "the target system for EDM-Account-Master has seen several out of memory alerts and disc space errors in the last six weeks." We would instruct the risk prompt to consider that environment when deriving its risk assessment which should enhance its predictions.
* **More Context - Incident Root Causes (Rear view Mirror)**: We could pass a synopsis of the last 6 months of root causes to the model. This would make it keenly aware \

# Create Prompt Function with Python

In [ ]:
def generate_risk_prediction_prompt(cr_data):
    prompt = f"""
            <role>
                You are an experienced dev ops professional deeply knowledgeable on computer systems that support a very large company and the metadata that is captured about change requests.
                A change request is a formal proposal for an alteration to the computer system that you manage.
                As a dev ops expert, you specialize in using the metadata provided about a change request to predict the liklihood of the change request unintentionally destabalizing the computer system.
                You are going to be provided with change request meta data as a json object held between <cr_data> and your job is to provide a prediction score and reasoning behind the risk score in the <output> section. 
            </role>
        
            <task>: Follow these instructions,
                1) Considering the <cr_data> and your <role>, provide a risk score between 0 to 5 of this change request destabalizing the computer system when deployed. do not exhibit a bias toward high risk. base your risk score only on the data you have been provided. if there is not enough information, please indicate this. Output this as [Risk_Score]. Then,
                2) Considering the <cr_data> and your <role>, provide a reasoning for the risk score in as few words as possible while maintaining all detail needed to understand your reasoning. Output this as [Risk_Score_Reason]
            </task>

            <meta_data>
                The folowing is the metadata for the cr_data. the metatdata follows this format: (column_name: data_type : description : sample_values):
                
                (Description: VARCHAR(16777216) : A summary of the change : ''Upgrade to the existing DataSync API from version 3.2 to 3.4 in the Production environment. The update includes several key performance optimizations, enhanced security features, and bug fixes that address issues with data consistency and processing time.
                The major components of this change include:
                
                API Version Update: Migrating from DataSync API v3.2 to v3.4 to support faster data ingestion and processing.
                Security Enhancements: Implementation of OAuth 2.0-based authentication to replace the legacy basic authentication mechanism, improving overall security for API transactions.
                Error Handling: Enhanced error codes and more descriptive responses for improved debugging in the event of failure.
                Database Schema Update: Modifications to the backend MySQL database to accommodate new data types introduced in version 3.4.
                Testing will be performed in the staging environment (v3.4-Stage) before deployment to ensure backwards compatibility with existing systems. No downtime is expected during the deployment, but a rollback plan has been prepared in case of critical issues.'',)
                   
                (Date: DATE : The date the request was made : 2025-02-03)
                
                (Impact: VARCHAR(4000) : How the change will affect the project, including cost, quality, risk, scope, duration, and schedule)
                
                (Priority: VARCHAR(400) : How quickly the change should be approved and implemented : ''immediately'', ''within release window'', ''after approval'')
                
                (Risk: VARCHAR(400) : The risk level of the change as described by development team : ''low risk'', ''moderate risk'', ''high risk'')
                
                (Justification: VARCHAR(4000) : The reason for the change : ''preventative maintenance'', ''patch'', ''planned release as part of project koala'')
                
                (State: VARCHAR(400) : The status of the change request : ''new'', ''under review'', ''approved'', ''deferred'', ''rejected'')
                
                (Disposition: VARCHAR(400) : An explanation for approved, deferred, or rejected changes : ''no peer review'', ''manager override'', ''uniform agreement'')
                
                (Category: VARCHAR(400) : The category of the change : ''planned'', ''unplanned'', ''emergency'')
                
                (Change number: VARCHAR(100) : A unique ID for tracking the request :''AB1672'', ''723CS'', ''6D62EE'')
            </meta_data>
            
            <cr_data>
                {cr_data}
            </cr_data>
        
            <Output> 
                produce valid JSON. Absolutely do not include any additional text before or following the JSON. Output should use following <JSON_format>
            </Output>
            
            <JSON_format>
            {{
                "Risk_Score": (A risk score between 0 to 5 of this change request destabalizing the computer system when deployed),
                "Risk_Score_Reason": (A concise resoning for the Risk_Score and any suggestions to mitigate),
            }}
            </JSON_format>
            """
    return prompt

# my_complete() 

We can call our LLMs via the LLM API (https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-llm-rest-api) or using SQL. To make passing our parameters easy, I am coing to use SQL and wrap it in a python function to call later.

In [ ]:
def my_complete(model, context, temp = 0, max_tokens: int = 18000):
    sql = F"""SELECT SNOWFLAKE.CORTEX.COMPLETE(
            '{model}',
            [
                {{
                    'role': 'user',
                    'content': '{context}'
                }}
            ],
            {{
                'max_tokens': {max_tokens}, 
                'temperature' : {temp}
            }}
        ) as inference;"""
    inference_raw = session.sql(sql).to_pandas().loc[0,"INFERENCE"]
    inference_json = json.loads(inference_raw)
    inference_raw = inference_json['choices'][0]['messages']
    return inference_raw

# Batch Inference
## Generate Risk Score and Reasoning



### 🤯 Whoa, check that out 🤯
In a single line of python (10), we compile our prompt and call our LLM for inference. We could swap ANY LLM WE WANT here with exactly 0 overhead. 

https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions#availability

In [ ]:
import streamlit as st

session = get_active_session()
database = 'GEN_AI_FSI'
schema = 'DTCC_HACKATHON'
table = 'CHANGE_REQUEST_RAW'

df = session.table(f"{database}.{schema}.{table}").to_pandas()

df['RISK_ASSESSMENT'] = df.apply(lambda row: my_complete('claude-3-5-sonnet', generate_risk_prediction_prompt(row.to_json())), axis = 1)

#caution-- this will generate inference for each row in teh dataset and may tame a moment to run. 
#You can also use df.head() to run this on a sample
session.write_pandas(df, table_name="CHANGE_REQUEST_RISK_INFERENCE", database="GEN_AI_FSI", schema="DTCC_HACKATHON", auto_create_table=True, overwrite=True)

# Make It Relevant with Streamlit

Built but not accessible is the same as not built at all, we need to expose this functionlity to the end user. We will use Streamlit in Snowflake to do that. 

We want to give the end user the abilility to interact with this data in two ways: 
1. First we want them to be able to see all of their change requests in a single place-- this would be great for a systematic or weekly review of changes. 
2. Give the end users a way to select a change request and use an LLM of their choosing to generate the risk score and reasoning.

**Note**: A combined and polished version of this can be found in 3_HARBINGER_UI.py in this repository.

In [ ]:
import streamlit as st
import json5 as json
import pandas as pd
from snowflake.snowpark.functions import col, call_udf
from snowflake.snowpark.context import get_active_session

# # Page configuration
# st.set_page_config(
#     page_title="Change Request Risk Assessment",
#     page_icon="🔍",
#     layout="wide",
#     initial_sidebar_state="expanded"
# )

# Add legend for risk score colors
st.sidebar.title('Risk Score Legend')

functionality = st.sidebar.selectbox('Select Functionlity', ("Summary", "Ad Hoc"))

legend_cols = st.sidebar.columns(3)
with legend_cols[0]:
    st.color_picker('Low Risk (0-2)', '#00FF00', disabled=True)
with legend_cols[1]:
    st.color_picker('Medium Risk (2-3)', '#FFFF00', disabled=True)
with legend_cols[2]:
    st.color_picker('High Risk (4-5)', '#FF0000', disabled=True)

# Custom CSS for elegant styling
st.write("""
<style>
    .stButton button {
        background-color: #4CAF50;
        color: white;
        padding: 0.5rem 1rem;
        border-radius: 5px;
        border: none;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        transition: all 0.3s ease;
    }
    .stButton button:hover {
        background-color: #45a049;
        box-shadow: 0 4px 8px rgba(0,0,0,0.2);
    }
    div[data-testid="stDecoration"] {
        background-image: none;
    }
</style>
""", unsafe_allow_html=True)

# Initialize session and get data
session = get_active_session()
database = 'GEN_AI_FSI'
schema = 'DTCC_HACKATHON'
table = 'CHANGE_REQUEST_RISK_INFERENCE'

def color_risk_score(val):
    """
    Returns a color based on the risk score value:
    - Low risk (0-1): Green
    - Medium risk (2-3): Yellow
    - High risk (4-5): Red
    """
    if pd.isna(val):
        return ''
    
    # Normalize value between 0 and 1
    normalized = float(val) / 10
    
    # Create RGB values for gradient
    if normalized < 0.2:  # Green to Yellow
        r = int(255 * (normalized / 0.2))
        g = 255
        b = 0
    else:  # Yellow to Red
        r = 255
        g = int(255 * (1 - (normalized - 0.2) / 0.5))
        b = 0
    
    return f'background-color: rgba({r}, {g}, {b}, 0.2)'

@st.cache_data
def load_data():
    df = session.table(f"{database}.{schema}.{table}").to_pandas()
    for i in range(len(df)):
        json_data = json.loads(df.at[i, 'RISK_ASSESSMENT'])
        df.at[i, 'RISK_SCORE_AI'] = json_data["Risk_Score"]
        df.at[i, 'RISK_REASON_AI'] = json_data["Risk_Score_Reason"]
    return df

# Load data
df = load_data()

# Prepare display tables with styling
display_table = df[['CHANGENUMBER', 'DATE', 'RISK_SCORE_AI', 'RISK_REASON_AI']]
display_table_detail = df[['CHANGENUMBER', 'DESCRIPTION', 'DATE', 'IMPACT', 
                          'PRIORITY', 'RISK', 'JUSTIFICATION', 'STATE', 
                          'DISPOSITION', 'CATEGORY', 'RISK_SCORE_AI', 'RISK_REASON_AI']]

# App header
st.title('Risk Assessment Dashboard')

# Add some basic metrics
st.subheader('Key Metrics')
metric_col1, metric_col2, metric_col3 = st.columns(3)

with metric_col1:
    avg_risk = df['RISK_SCORE_AI'].mean()
    st.metric('Average Risk Score', f'{avg_risk:.2f}')

with metric_col2:
    high_risk_count = len(df[df['RISK_SCORE_AI'] > 3])
    st.metric('High Risk Changes', high_risk_count)

with metric_col3:
    total_changes = len(df)
    st.metric('Total Changes', total_changes)

# Main content area
col1, col2 = st.columns([3, 1])

title = 'Risk Assessment Summary'

st.subheader(title)
show_details = st.button('Show Detailed View', use_container_width=True)
    
# Show detailed view if button is clicked
if show_details:
    title = 'Detailed Risk Assessment'
    st.dataframe(
        display_table_detail.style.applymap(
            color_risk_score,
            subset=['RISK_SCORE_AI']
        ),
        use_container_width=True,
        hide_index=True,
        height=400
)
else: 
    st.dataframe(
        display_table.style.applymap(
            color_risk_score,
            subset=['RISK_SCORE_AI']
        ),
        use_container_width=True,
        hide_index=True,
        height=400
    )

In [ ]:
import streamlit as st
import json5 as json
import pandas as pd
from snowflake.snowpark.functions import col, call_udf
from snowflake.snowpark.context import get_active_session

# Page configuration
# st.set_page_config(
#     page_title="Change Request Risk Assessment",
#     page_icon="🔍",
#     layout="wide",
#     initial_sidebar_state="expanded"
# )

# Add legend for risk score colors
st.sidebar.title('Risk Score Legend')

functionality = st.sidebar.selectbox('Select Functionlity', ("Summary", "Ad Hoc"))

legend_cols = st.sidebar.columns(3)
with legend_cols[0]:
    st.color_picker('Low Risk (0-2)', '#00FF00', disabled=True)
with legend_cols[1]:
    st.color_picker('Medium Risk (2-3)', '#FFFF00', disabled=True)
with legend_cols[2]:
    st.color_picker('High Risk (4-5)', '#FF0000', disabled=True)

##########################
##########AD HOC##########
##########################

# Custom CSS to enhance the UI
st.markdown("""
    <style>
    .risk-score {
        font-size: 24px;
        font-weight: bold;
        padding: 1rem;
        border-radius: 8px;
        margin: 1rem 0;
    }
    .info-box {
        background-color: #f8f9fa;
        padding: 1.5rem;
        border-radius: 8px;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }
    </style>
    """, unsafe_allow_html=True)

# Header with gradient background
st.title("Change Request Risk Assessment")

# Initialize session
session = get_active_session()

def generate_risk_prediction_prompt(cr_data):
    prompt = f"""
            <role>
            You are an experienced dev ops professional deeply knowledgeable on computer systems that support a very large company and the metadata that is captured about change requests.
            A change request is a formal proposal for an alteration to the computer system that you manage.
            As a dev ops expert, you specialize in using the metadata provided about a change request to predict the liklihood of the change request unintentionally destabalizing the computer system.
            You are going to be provided with change request meta data as a json object held between <cr_data> and your job is to provide a prediction score and reasoning behind the risk score in the <output> section. 
            </role>
        
            <task>: Follow these instructions,
            1) Considering the <cr_data> and your <role>, provide a risk score between 0 to 5 of this change request destabalizing the computer system when deployed. do not exhibit a bias toward high risk. base your risk score only on the data you have been provided. if there is not enough information, please indicate this. Output this as [Risk_Score]. Then,
            2) Considering the <cr_data> and your <role>, provide a reasoning for the risk score in as few words as possible while maintaining all detail needed to understand your reasoning. Output this as [Risk_Score_Reason]
            </task>

            <cr_data>
            {cr_data}
            </cr_data>
        
            <Output> 
            produce valid JSON. Absolutely do not include any additional text before or following the JSON. Output should use following <JSON_format>
            </Output>
            
            <JSON_format>
            {{
                "Risk_Score": (A risk score between 0 to 5 of this change request destabalizing the computer system when deployed),
                "Risk_Score_Reason": (A concise resoning for the Risk_Score and any suggestions to mitigate),
            }}
            </JSON_format>
            """
    return prompt

def my_complete(model, context, temp=0, max_tokens: int = 18000):
    sql = F"""SELECT SNOWFLAKE.CORTEX.COMPLETE(
            '{model}',
            [
                {{
                    'role': 'user',
                    'content': '{context}'
                }}
            ],
            {{
                'max_tokens': {max_tokens}, 
                'temperature' : {temp} 
            }}
        ) as inference;"""
    inference_raw = session.sql(sql).to_pandas().loc[0,"INFERENCE"]
    inference_json = json.loads(inference_raw)
    inference_raw = inference_json['choices'][0]['messages']
    return inference_raw

# Database configuration
database = 'GEN_AI_FSI'
schema = 'DTCC_HACKATHON'
table = 'ChangeRequests'
history_table = 'CHANGE_REQUEST_RISK_INFERENCE'

# Get the data into pandas
cr_df = session.table(f"{database}.{schema}.{table}")

# Create two columns for the input section
col1, col2 = st.columns(2)

with col1:
    #st.markdown('<div class="info-box">', unsafe_allow_html=True)
    cr_request = st.selectbox(
        'Select a Change Request',
        cr_df,
        help="Choose the change request you want to analyze"
    )
    st.markdown('</div>', unsafe_allow_html=True)

with col2:
    #st.markdown('<div class="info-box">', unsafe_allow_html=True)
    # Available models
    models = [
        'snowflake-arctic', 'claude-3-5-sonnet', 'mistral-large',
        'mistral-large2', 'reka-flash', 'reka-core', 'jamba-instruct',
        'jamba-1.5-mini', 'jamba-1.5-large', 'mixtral-8x7b',
        'llama2-70b-chat', 'llama3-8b', 'llama3-70b', 'llama3.1-8b',
        'llama3.1-70b', 'llama3.3-70b', 'snowflake-llama-3.3-70b',
        'llama3.1-405b', 'snowflake-llama-3.1-405b', 'llama3.2-1b',
        'llama3.2-3b', 'mistral-7b', 'gemma-7b'
    ]
    
    user_input_model = st.selectbox(
        "Select AI Model",
        models,
        help="Choose the AI model for risk assessment",
        key="CS_model_select_box"
    )
    st.markdown('</div>', unsafe_allow_html=True)

# Add a loading spinner
with st.spinner("Analyzing risk..."):
    df = session.table(f"{database}.{schema}.{table}").filter(col("CHANGENUMBER") == cr_request).to_pandas()
    
    if st.button("Analyze Risk", type="primary"):
        try:
            df['RISK_ASSESSMENT'] = df.apply(
                lambda row: my_complete(user_input_model, generate_risk_prediction_prompt(row.to_json())),
                axis=1
            )
            
            json_data = json.loads(df.at[0, 'RISK_ASSESSMENT'])
            
            # Create three columns for the results
            result_col1, result_col2, result_col3 = st.columns([1,2,1])
            
            with result_col1:
                #st.markdown('<div class="info-box">', unsafe_allow_html=True)
                # Color code the risk score
                risk_score = float(json_data["Risk_Score"])
                color = "green" if risk_score <= 2 else "orange" if risk_score <= 3.5 else "red"
                st.markdown(f'<div class="risk-score" style="background-color: {color}; color: white;">'
                          f'Risk Score: {risk_score}</div>', unsafe_allow_html=True)
                st.markdown('</div>', unsafe_allow_html=True)
            
            with result_col2:
                #st.markdown('<div class="info-box">', unsafe_allow_html=True)
                st.subheader("Risk Assessment")
                st.write(json_data["Risk_Score_Reason"])
                st.markdown('</div>', unsafe_allow_html=True)
            
            with result_col3:
                with st.expander("View Raw JSON"):
                #if st.button("View Raw JSON"):
                    st.json(json_data)

        except ValueError as e:
            st.error(f"Error: {user_input_model} did not produce valid output. Please select another model.")
            st.exception(e)
        except Exception as e:
            st.error("An unexpected error occurred. Please try again.")
            st.exception(e)
